In [24]:
import os
import numpy as np
import torch
from collections import OrderedDict
import json

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
query_feature_A_dir = '/nfs3-p2/zsxm/naic/preliminary/test_A/query_feature_A'
query_feature_A = []
for data in sorted(os.listdir(query_feature_A_dir)):
    query_feature_A.append(torch.from_numpy(np.fromfile(os.path.join(query_feature_A_dir, data), dtype='<f4')))
query_feature_A = torch.stack(query_feature_A).to(device)

In [3]:
gallery_feature_A_dir = '/nfs3-p2/zsxm/naic/preliminary/test_A/gallery_feature_A'
gallery_feature_A = []
for data in sorted(os.listdir(gallery_feature_A_dir)):
    gallery_feature_A.append(torch.from_numpy(np.fromfile(os.path.join(gallery_feature_A_dir, data), dtype='<f4')))
gallery_feature_A = torch.stack(gallery_feature_A).to(device)

In [4]:
print(query_feature_A.shape, query_feature_A)
print(gallery_feature_A.shape, gallery_feature_A)

torch.Size([20000, 2048]) tensor([[ 0.0805,  0.0000,  0.0000,  ...,  0.0000,  0.0646,  0.0000],
        [-0.2679,  0.0000,  0.0000,  ...,  0.0000, -0.2808,  0.0000],
        [-0.3382,  0.0000,  0.0000,  ...,  0.0000,  0.0906,  0.0000],
        ...,
        [-0.2313,  0.0000,  0.0000,  ...,  0.0000, -0.4064,  0.0000],
        [-0.1418,  0.0000,  0.0000,  ...,  0.0000,  0.0699,  0.0000],
        [ 0.0150,  0.0000,  0.0000,  ...,  0.0000, -0.2349,  0.0000]],
       device='cuda:0')
torch.Size([428794, 2048]) tensor([[ 0.1381,  0.0000,  0.0000,  ...,  0.0000, -0.0197,  0.0000],
        [-0.3590,  0.0000,  0.0000,  ...,  0.0000, -0.5844,  0.0000],
        [-0.3214,  0.0000,  0.0000,  ...,  0.0000, -0.2519,  0.0000],
        ...,
        [-0.0717,  0.0000,  0.0000,  ...,  0.0000, -0.4211,  0.0000],
        [-0.5035,  0.0000,  0.0000,  ...,  0.0000,  0.2123,  0.0000],
        [-0.2045,  0.0000,  0.0000,  ...,  0.0000, -0.6521,  0.0000]],
       device='cuda:0')


In [12]:
def cos_similarity(q, k):
    try:
        mm = torch.mm(q, k.T)
        qn = torch.linalg.vector_norm(q, dim=1, keepdim=True)
        kn = torch.linalg.vector_norm(k, dim=1, keepdim=True)
        qk = torch.mm(qn, kn.T)
        res = mm / qk
    except RuntimeError:
        q = q.cpu()
        k = k.cpu()
        mm = torch.mm(q, k.T)
        qn = torch.linalg.vector_norm(q, dim=1, keepdim=True)
        kn = torch.linalg.vector_norm(k, dim=1, keepdim=True)
        qk = torch.mm(qn, kn.T)
        res = mm / qk
    return res

In [13]:
res = cos_similarity(query_feature_A, gallery_feature_A)

In [14]:
print(res.shape)

torch.Size([20000, 428794])


In [22]:
res_dict = {}
query_names = sorted(os.listdir(query_feature_A_dir))
gallery_names = sorted(os.listdir(gallery_feature_A_dir))
for i, name in enumerate(query_names):
    idx = torch.argsort(res[i], dim=-1, descending=True)
    query_res = []
    for j in range(100):
        query_res.append(gallery_names[idx[j]])
    res_dict[name] = query_res

In [25]:
with open('./sub_a.json', 'w') as f:
    json.dump(res_dict, f)